In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re

import cptac
import cptac.utils as u


In [2]:
b = cptac.Gbm()

Transcriptomics

Name                  5S_rRNA                                      \
Database_ID ENSG00000201285.1 ENSG00000212595.1 ENSG00000252830.2   
Sample_ID                                                           
S001                      0.0               0.0               0.0   
S002                      0.0               0.0               0.0   
S003                      0.0               0.0               0.0   
S004                      0.0               0.0               0.0   
S005                      0.0               0.0               0.0   
...                       ...               ...               ...   
S111                      0.0               0.0               0.0   
S112                      0.0               0.0               0.0   
S113                      0.0               0.0               0.0   
S114                      0.0               0.0               0.0   
S115                      0.0               0.0               0.0   

Name                                                               \
Database_ID ENSG00000271924.1 ENSG00000272253.1 ENSG00000272351.1   
Sample_ID                                                           
S001             94745.135585      10905.244332            0.0000   
S002             49830.181008      24766.912396            0.0000   
S003             61838.821529      28979.207419         4342.8224   
S004             65854.771413       5143.530211            0.0000   
S005             60360.229852      33000.686414            0.0000   
...                       ...               ...               ...   
S111             14677.175524      10699.249448            0.0000   
S112             30616.150019      16738.736225            0.0000   
S113             58719.906048          0.000000            0.0000   
S114             20979.770257       3277.213778            0.0000   
S115              8385.586016       9169.285644            0.0000   

Name                                                               \
Database_ID ENSG00000272435.1 ENSG00000274059.1 ENSG00000274097.1   
Sample_ID                                                           
S001                      0.0          0.000000          0.000000   
S002                      0.0          0.000000       4953.382479   
S003                      0.0          0.000000          0.000000   
S004                      0.0      13103.755536          0.000000   
S005                      0.0          0.000000          0.000000   
...                       ...               ...               ...   
S111                      0.0          0.000000          0.000000   
S112                      0.0          0.000000          0.000000   
S113                      0.0          0.000000          0.000000   
S114                      0.0          0.000000          0.000000   
S115                      0.0          0.000000          0.000000   

Name                           ...            uc_338                    \
Database_ID ENSG00000274164.1  ... ENSG00000278040.1 ENSG00000278113.1   
Sample_ID                      ...                                       
S001                 0.000000  ...      17364.005112               0.0   
S002                 0.000000  ...      22083.830220               0.0   
S003                 0.000000  ...      21533.161068               0.0   
S004             13315.106432  ...      16379.694420               0.0   
S005                 0.000000  ...      42036.588647               0.0   
...                       ...  ...               ...               ...   
S111                 0.000000  ...      34072.014610               0.0   
S112              4814.636495  ...      28429.282161               0.0   
S113                 0.000000  ...      18175.209015               0.0   
S114                 0.000000  ...      98101.833878               0.0   
S115                 0.000000  ...     112905.926004               0.0   

Name                                                          

In [58]:
omics_name = 'transcriptomics'
mutations_df = b.get_somatic_mutation()
all_genes_list = list(mutations_df['Gene'])


In [73]:
# get tumor sample ids
omics_and_mut = b.join_omics_to_mutations( 
    omics_df_name=omics_name, 
    mutations_genes='PTEN',
    omics_genes='PTEN')

# drop normal samples
omics_and_mut = b.reduce_multiindex(omics_and_mut, levels_to_drop = 1,flatten=True)
tumors = omics_and_mut.loc[omics_and_mut['Sample_Status'] == 'Tumor']
tumor_sample_id_list = list(tumors.index)
omics_and_mut

Name,PTEN_transcriptomics,PTEN_Mutation,PTEN_Location,PTEN_Mutation_Status,Sample_Status
Sample_ID,,,,,
S001,177100.345056,[Missense_Mutation],[p.G251D],Single_mutation,Tumor
S002,175155.715999,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
S003,349109.743565,[Missense_Mutation],[p.C136Y],Single_mutation,Tumor
S004,207316.199536,[Missense_Mutation],[p.R173H],Single_mutation,Tumor
S005,213887.007588,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
...,...,...,...,...,...
S111,275161.172491,[Wildtype_Normal],[No_mutation],Wildtype_Normal,Normal
S112,341202.579113,[Wildtype_Normal],[No_mutation],Wildtype_Normal,Normal
S113,257442.126251,[Wildtype_Normal],[No_mutation],Wildtype_Normal,Normal


In [61]:
# test transcriptomics
clin_and_omics = b.join_metadata_to_omics( 
    omics_df_name=omics_name, 
    metadata_df_name="clinical",
    metadata_cols = ["gender"], 
    omics_genes=all_genes_list)

clin_and_omics = b.reduce_multiindex(clin_and_omics, levels_to_drop = 1,flatten=True)

# drop Normal samples FIX
tumors_bool = clin_and_omics.index.isin(tumor_sample_id_list)
tumor_clin_and_omics = clin_and_omics[tumors_bool]
tumor_clin_and_omics

Name,gender,A2ML1_transcriptomics,A4GALT_transcriptomics,AACS_transcriptomics,AADACL4_transcriptomics,AAED1_transcriptomics,AASS_transcriptomics,AATK_transcriptomics,ABCA12_transcriptomics,ABCA13_transcriptomics,...,ZSCAN18_transcriptomics,ZSCAN22_transcriptomics,ZSCAN5B_transcriptomics,ZSCAN5C_transcriptomics,ZSCAN9_transcriptomics,ZSWIM4_transcriptomics,ZW10_transcriptomics,ZWILCH_transcriptomics,ZXDA_transcriptomics,ZZEF1_transcriptomics
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,Male,21010.850387,15661.235192,42957.948772,0.000000,103292.182938,400306.316935,73854.343863,3098.080776,157435.102908,...,266817.472267,27602.353687,566.712551,308.367110,95584.981662,50630.830437,211571.581003,40547.762979,67662.626484,163807.374286
S002,Female,37521.041403,6663.418700,23283.113655,0.000000,130612.536707,628602.686362,29203.869938,2758.133426,6391.042404,...,113824.638820,56791.081213,2059.298398,280.133153,123168.763200,60839.476251,204954.477786,65909.917702,93554.256685,228521.327647
S003,Male,11110.085020,33715.530480,30794.534307,0.000000,130371.979073,402990.459370,161555.783859,2908.897714,36252.116400,...,131889.639478,55598.259038,501.987242,0.000000,128885.122765,52414.263478,216598.267214,95173.996290,55537.022273,202252.771699
S004,Female,23211.034043,27489.835773,40454.405218,0.000000,94581.581790,490658.723422,84078.747571,4091.444486,13388.171746,...,75527.605042,54096.515079,801.881106,0.000000,151069.855325,98907.914163,311231.712378,49485.661246,74264.172514,253584.656201
S005,Female,21177.737498,41513.809461,43661.216343,0.000000,78315.636747,573001.463192,73968.189141,6225.129483,19623.245534,...,137213.254902,46684.315169,1028.967493,373.263578,142005.382862,68928.231925,206057.962476,44857.522024,63530.132514,228214.357641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S100,Male,75389.259173,97072.363156,45656.631980,0.000000,140671.891347,499017.671647,11797.162118,5680.347378,119442.976444,...,388365.087864,35764.176249,412.329540,0.000000,162693.773512,59084.994553,205436.578267,62592.840119,78334.642652,203958.879548
S101,Male,31374.126867,13172.747410,31345.418232,0.000000,76551.069717,915223.102682,21397.718109,1794.669536,15868.926110,...,122713.819407,53685.233597,965.552001,0.000000,158160.349643,77194.376706,208973.432134,83229.228465,76129.655328,171630.501661
S102,Female,3172.409071,23433.746939,60159.761830,684.181719,97660.617648,420495.684961,126107.879795,3719.366156,64517.324540,...,257494.717791,44695.733792,2093.416623,0.000000,117938.247475,49157.868551,191602.895077,59112.792542,58293.929257,226131.580890


In [69]:
omics_col = tumor_clin_and_omics.columns
omics_col_list = list(omics_col[1:])
omics_col_list

['A2ML1_transcriptomics',
 'A4GALT_transcriptomics',
 'AACS_transcriptomics',
 'AADACL4_transcriptomics',
 'AAED1_transcriptomics',
 'AASS_transcriptomics',
 'AATK_transcriptomics',
 'ABCA12_transcriptomics',
 'ABCA13_transcriptomics',
 'ABCA3_transcriptomics',
 'ABCA4_transcriptomics',
 'ABCA5_transcriptomics',
 'ABCA7_transcriptomics',
 'ABCA8_transcriptomics',
 'ABCA9_transcriptomics',
 'ABCB1_transcriptomics',
 'ABCB11_transcriptomics',
 'ABCB5_transcriptomics',
 'ABCC10_transcriptomics',
 'ABCC11_transcriptomics',
 'ABCC12_transcriptomics',
 'ABCC3_transcriptomics',
 'ABCC4_transcriptomics',
 'ABCC6_transcriptomics',
 'ABCC9_transcriptomics',
 'ABCD1_transcriptomics',
 'ABCF1_transcriptomics',
 'ABCG4_transcriptomics',
 'ABCG8_transcriptomics',
 'ABHD13_transcriptomics',
 'ABHD6_transcriptomics',
 'ABI3BP_transcriptomics',
 'ABL1_transcriptomics',
 'ABR_transcriptomics',
 'ABRA_transcriptomics',
 'AC005833.1_transcriptomics',
 'AC007040.2_transcriptomics',
 'AC011604.2_transcripto

In [66]:
u.wrap_ttest(tumor_clin_and_omics, 'gender', omics_col_list)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Incorrectly Formatted Dataframe!
